#### 【문항01】 경찰 민원포탈 > 경찰관서 찾기 > 목록에서 찾기 > 서울
          에서 서울 소재 31개의 경찰서의 이름과 주소를 크롤링하여 데이터프레임을 만드세요. (40점)
          (단, 셀레니움을 이용해서 가져오지 않고 다른 사이트를 이용해서 가져오면 감점 20점)

In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import warnings
warnings.filterwarnings('ignore')

- 1-1. 데이터 가져오기 (10점)

In [3]:
url = 'https://minwon.police.go.kr/#customerCenter/plcstSearch'
chromedriver = '../05.Crawling/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)
driver.get(url)

- 1-2. 원하는 데이터를 찾고, 하나를 선택해서 원하는 정보 추출하기 (10점)

In [5]:
driver.find_element_by_id('tab3').click()
driver.find_element_by_id('indexedA').click()

In [6]:
office = driver.find_element_by_xpath('//*[@id="minwonTbody"]/tr[1]/td[1]')
office.text

'경찰청 서울특별시경찰청 서울중부경찰서'

In [7]:
path = driver.find_element_by_xpath('//*[@id="minwonTbody"]/tr[1]/td[2]')
path.text                               # page7 //*[@id="minwonTbody"]/tr

'서울특별시 중구 수표로 27'

- 1-3. 반복문을 사용하여 모든 페이지의 데이터 가져오기 및 오류 수정하기 (10점)

In [8]:
# page7에 갯수 1개
len(driver.find_element_by_id('minwonTbody').find_elements_by_tag_name('tr'))

5

In [9]:
from tqdm.notebook import tqdm
lines = []
url = 'https://minwon.police.go.kr/#customerCenter/plcstSearch'
chromedriver = '../05.Crawling/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)
driver.get(url)
time.sleep(4)
driver.find_element_by_id('tab3_1').click()                           # 목록에서 찾기 클릭
time.sleep(2)
driver.find_element_by_id('indexedA').click()                       # 서울 클릭
time.sleep(2)
for i in tqdm(range(7)):
    if i > 0:
        driver.find_element_by_xpath(f'//*[@id="pagination"]/div/a[{i+2}]').click()
        time.sleep(2)

    list_num = len(driver.find_element_by_id('minwonTbody').find_elements_by_tag_name('tr'))   # 목록 갯수 예외상황
    
    if list_num == 1:
        office = driver.find_element_by_xpath(f'//*[@id="minwonTbody"]/tr/td[1]').text
        path = driver.find_element_by_xpath(f'//*[@id="minwonTbody"]/tr/td[2]').text
        lines.append([office, path])
    else:
        for j in range(list_num):
            office = driver.find_element_by_xpath(f'//*[@id="minwonTbody"]/tr[{j+1}]/td[1]').text
            path = driver.find_element_by_xpath(f'//*[@id="minwonTbody"]/tr[{j+1}]/td[2]').text
            lines.append([office, path])

  0%|          | 0/7 [00:00<?, ?it/s]

- 1-4. 데이터 프레임으로 만들고 저장하기 (10점)

In [11]:
df = pd.DataFrame(lines, columns=['관서','주소'])

In [12]:
# 경찰청 서울특별시 없애기
df.관서 = df.관서.str.split().str[2]

In [13]:
# 홈페이지 오류
print(df[df.관서 == '서울서부경찰서'], df[df.관서 == '서울강동경찰서'], sep='\n')

         관서                  주소
13  서울서부경찰서  서울특별시 은평구 은평로9길 15
         관서                주소
20  서울강동경찰서  서울특별시 강동구 성내로 33


In [14]:
df['주소'][13] = '서울특별시 은평구 서오릉로 58'
df['주소'][20] = '서울 강동구 양재대로96길 35'
print(df[df.관서 == '서울서부경찰서'], df[df.관서 == '서울강동경찰서'], sep='\n')

         관서                 주소
13  서울서부경찰서  서울특별시 은평구 서오릉로 58
         관서                 주소
20  서울강동경찰서  서울 강동구 양재대로96길 35


In [19]:
df.head()

,관서,주소
0,서울중부경찰서,서울특별시 중구 수표로 27
1,서울종로경찰서,서울특별시 종로구 율곡로 46
2,서울남대문경찰서,서울특별시 중구 한강대로 410
3,서울서대문경찰서,서울특별시 서대문구 통일로 113
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16


- 1-4. 데이터 프레임으로 만들고 저장하기 (10점)

In [15]:
df.to_csv('서울 경찰서.csv', index=False)

#### 【문항02】1번에서 구한 경찰서의 도로명 주소를 가지고, 카카오로컬 API를 이용하여 경찰서의 위도, 경도
        정보를 구한 후 데이터프레임에 추가하세요. (30점)

- 2-1. 카카오 로컬 API 사이트에 접속하기 (10점)

In [20]:
from urllib.parse import quote
import requests


In [21]:
with open('../04.지도시각화/kakaoapikey.txt') as file:
    kakao_key = file.read()

In [23]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
addr = '서울특별시 중구 수표로 27'
url = f'{local_url}?query={quote(addr)}'
header = {'Authorization': f'KakaoAK {kakao_key}'}
result = requests.get(url, headers=header).json()


In [25]:
lng = float(result['documents'][0]['x'])
lat = float(result['documents'][0]['y'])
print(lat, lng)

37.5636321301236 126.990020639754


- 2-2. 반복문을 사용하여 모든 경찰서의 위도, 경도 좌표 구하기 (10점)

In [26]:
police = pd.read_csv('서울 경찰서.csv')
police.head()

,관서,주소
0,서울중부경찰서,서울특별시 중구 수표로 27
1,서울종로경찰서,서울특별시 종로구 율곡로 46
2,서울남대문경찰서,서울특별시 중구 한강대로 410
3,서울서대문경찰서,서울특별시 서대문구 통일로 113
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16


In [27]:
len(police)

31

In [28]:
lng_list, lat_list = [], []
for i in police.index:
    url = f'{local_url}?query={quote(police.주소[i])}'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))

- 2-3. 데이터프레임에 위도, 경도 컬럼 추가하기 (10점)

In [29]:
police['위도'] = lat_list
police['경도'] = lng_list


In [30]:
police.head()

,관서,주소,위도,경도
0,서울중부경찰서,서울특별시 중구 수표로 27,37.563632,126.990021
1,서울종로경찰서,서울특별시 종로구 율곡로 46,37.575876,126.984551
2,서울남대문경찰서,서울특별시 중구 한강대로 410,37.554811,126.973549
3,서울서대문경찰서,서울특별시 서대문구 통일로 113,37.565085,126.967074
4,서울혜화경찰서,서울특별시 종로구 창경궁로 112-16,37.571768,126.999273


#### 【문항03】 1,2번에서 구한 정보를 바탕으로 서울 지도(Open Street Map)위에 표시하세요. (30점)

In [50]:
# 3-1. 서울 지도를 그릴 것 (10점)
import folium
map = folium.Map(
    location=[police.위도.mean(), police.경도.mean()], zoom_start=12
)
# 3-2. 툴팁에는 경찰서명을 표시하고, 팝업에는 경찰서 주소를 표시하되 팝업창의 너비를 늘려 가로로 표시할 것 (10점)
for i in police.index:
    folium.Marker(
        location=[police.위도[i], police.경도[i]],
        popup=folium.Popup(police.주소[i], max_width=200),
        tooltip=police.관서[i],
        icon=folium.Icon(color='blue', icon='glyphicon glyphicon-tower')
    ).add_to(map)
map

# 3-3. 아이콘은 기본 모양이 아닌 것으로 표시하고, 지도 제목을 ‘서울소재 경찰서 위치’로 표시할 것 (10점)
title_html = '<h3 align="center" style="font-size:20px">서울소재 경찰서 위치</h3>'
map.get_root().html.add_child(folium.Element(title_html))

map